In [1]:
import pandas as pd
import numpy as np

def optimized_function_maid_vectorized(df, lat_col='latitude', lon_col='longitude', datetime_col='datetime', maid_col='maid'):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values(by=[maid_col, datetime_col, lat_col, lon_col])

    # Shift columns to get previous values for each maid
    df['prev_X'] = df.groupby(maid_col)[lon_col].shift(1)
    df['prev_Y'] = df.groupby(maid_col)[lat_col].shift(1)
    df['prev_datetime'] = df.groupby(maid_col)[datetime_col].shift(1)

    # Radius of the Earth in kilometers
    R = 6371

    # Convert latitude and longitude from degrees to radians
    lat1 = np.radians(df[lat_col].values)
    lat2 = np.radians(df['prev_Y'].values)
    dlat = lat2 - lat1
    dlon = np.radians(df['prev_X'].values) - np.radians(df[lon_col].values)

    # Haversine formula for distance calculation
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate consecutive distance in kilometers
    df['displacement'] = R * c
    df['distance'] = df.groupby(maid_col)['displacement'].cumsum()

    # Calculate consecutive time gap in minutes
    df['consecutive_time_gap(minutes)'] = (df[datetime_col] - df['prev_datetime']).dt.total_seconds() / 60

    # Calculate velocity
    df['Velocity'] = df['displacement'] / (df['consecutive_time_gap(minutes)'] / 60)

    # Drop temporary columns
    df = df.drop(columns=['prev_X', 'prev_Y', 'prev_datetime', 'consecutive_time_gap(minutes)'])

    # Fill NaN values with 0
    df = df.fillna(0)

    return df


In [2]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
from tqdm import tqdm

def process_dataframe_for_fixing_velocity_recursive(df, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90):
    """
    Process the DataFrame to find and flag records based on given conditions, and adjust datetimes if needed.

    Args:
    - df (DataFrame): The input DataFrame with data.
    - velocity_threshold (float): The threshold for velocity to consider a record.
    - displacement_threshold (float): The threshold for displacement to consider a record.
    - min_velocity (float): The minimum velocity for the ratio of distance to time difference.
    - max_velocity (float): The maximum velocity for the ratio of distance to time difference.

    Returns:
    - DataFrame: The processed DataFrame with records flagged for keeping or removal.
    """
    # Ensure the datetime column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Add new columns for actions chosen and keep flag
    df['Action chosen'] = 'None'
    df['keep'] = True

    # Calculate geodesic distance
    def calculate_geodesic_distance(lat1, lon1, lat2, lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).kilometers

    # Calculate time difference in hours
    def calculate_time_diff_hours(time1, time2):
        return abs((time1 - time2).total_seconds()) / 3600

    # Function to process each group
    def process_group(group):
        max_distance_transition_while_tracing = 20
        # Reset index for group processing
        group = group.reset_index(drop=True)
        
        # Filter relevant records based on velocity and displacement thresholds
        relevant_indices = group[(group['Velocity'] > velocity_threshold) & (group['displacement'] > displacement_threshold)].index
        count = len(relevant_indices)
        # print(f'Group size: {len(group)}, Relevant records count: {count}')
        
        # Process each relevant record
        for i in range(count):
            index = relevant_indices[i]
            record = group.iloc[index]
            # print(f'\nProcessing record at index {index}: {record.to_dict()}')
            
            # Set initial velocity range
            current_min_velocity = min_velocity
            current_max_velocity = max_velocity
            
            # Check distance from airport
            if index > 0:
                distance_from_airport_i = group.loc[index, 'Distance to airport (km)']
                distance_from_airport_prev = group.loc[index-1, 'Distance to airport (km)']
                # print(f'Distance from airport (current): {distance_from_airport_i}, (previous): {distance_from_airport_prev}')
                
                if distance_from_airport_i < 2 and distance_from_airport_prev < 2:
                    current_min_velocity = 500
                    current_max_velocity = 600
                    max_distance_transition_while_tracing = 2
                    # print(f'Updated velocity range: {current_min_velocity} - {current_max_velocity}')
            
            valid_record_found = False
            # Iterate backwards from index i to find a valid previous record
            for l in range(index - 1, -1, -1):
                if index - l > 1:
                    distance_change = calculate_geodesic_distance(
                        group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                        group.loc[l + 1, 'latitude'], group.loc[l + 1, 'longitude']
                    )
                    # print(f'Distance change between index {l} and {l+1}: {distance_change} km')
                    
                    if distance_change < max_distance_transition_while_tracing:
                        distance = calculate_geodesic_distance(
                            group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                            group.loc[l, 'latitude'], group.loc[l, 'longitude']
                        )
                        time_diff_hours = calculate_time_diff_hours(
                            group.loc[index, 'datetime'], group.loc[l, 'datetime']
                        )
                        # print(f'Distance to previous record: {distance} km, Time difference: {time_diff_hours} hours')
                        
                        if time_diff_hours > 0 and distance >= 0:
                            velocity_ratio = distance / time_diff_hours
                            # print(f'Calculated velocity ratio: {velocity_ratio}')
                        elif time_diff_hours == 0 and distance ==0:
                            velocity_ratio = 0
                            # print(f'Calculated velocity ratio: {velocity_ratio}')                            
                        elif time_diff_hours== 0:
                            velocity_ratio = np.inf
                            # 
                        # print(f'Calculated velocity ratio: {velocity_ratio}')     


                        if current_min_velocity < velocity_ratio < current_max_velocity:
                            group.at[l, 'Action chosen'] = 'selected'
                            group.loc[l + 1:index-1, 'keep'] = False
                            valid_record_found = True
                            # print(f'Record at index {l} selected.')
                            break

                        elif velocity_ratio < current_min_velocity:
                            # Calculate time x for comparison
                            distance_l_l1 = calculate_geodesic_distance(
                                group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                                group.loc[l, 'latitude'], group.loc[l, 'longitude']
                            )
                            time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)
                            # print(f'Time x for record at index {l+1}: {time_x}')

                            # Calculate new time for record l
                            distance_l_plus_1_index = calculate_geodesic_distance(
                                group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                                group.loc[index, 'latitude'], group.loc[index, 'longitude']
                            )
                            new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)
                            # print(f'New time for record at index {l+1}: {new_time}')

                            if new_time >= time_x:
                                group.at[l+1, 'datetime'] = new_time
                                group.at[l+1, 'Action chosen'] = 'modified'
                                
                                # Mark records from l+2 to index-1 for removal
                                group.loc[l + 2:index-1, 'keep'] = False
                                valid_record_found = True
                                # print(f'Record at index {l+1} modified.')
                                break
                    elif distance_change >= max_distance_transition_while_tracing:
                        # print(f'when distance>=20')
                        distance_l_l1 = calculate_geodesic_distance(
                            group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                            group.loc[l, 'latitude'], group.loc[l, 'longitude']
                        )
                        time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)
                        # print(f'Time x for record at index {l+1}: {time_x}')

                        distance_l_plus_1_index = calculate_geodesic_distance(
                            group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                            group.loc[index, 'latitude'], group.loc[index, 'longitude']
                        )
                        new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)
                        # print(f'New time for record at index {l+1}: {new_time}')
                        if new_time >= time_x:
                            group.at[l+1, 'datetime'] = new_time
                            group.at[l+1, 'Action chosen'] = 'modified'
                            
                            # Mark records from l+2 to index-1 for removal
                            group.loc[l + 2:index-1, 'keep'] = False
                            valid_record_found = True
                            # print(f'Record at index {l+1} modified.')
                            break
                        else:
                            group.at[index, 'Action chosen'] = 'Remained same'
                            break


            if not valid_record_found:
                # If no valid modification was made, mark as 'Remained same'
                group.at[index, 'Action chosen'] = 'Remained same'
                # print(f'Record at index {index} remained the same.')

        return group

    grouped = df.groupby('maid')
    # Process each group with a progress bar
    processed_groups = []
    for _, group in tqdm(grouped, total=len(grouped), desc="Processing Groups"):
        processed_groups.append(process_group(group))

    # Combine the processed groups back into a single DataFrame
    df = pd.concat(processed_groups)

    return df


In [3]:
# Define the column names
# Define the column names
columns = [
    'maid', 'datetime', 'latitude', 'longitude', 'displacement',
    'distance', 'Velocity', 'keep', 'Nearest airport', 'Coordinates of airport',
    'Distance to airport (km)', 'Added', 'Invalid', 'Unnamed: 13', 'Action chosen'
]

# Define the data
data = [
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-10 08:10:37", 28.6327, 77.2198, 0, 226.0821034, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-10 12:04:59", 28.6327, 77.2198, 0, 226.0821034, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-10 14:10:52", 28.6327, 77.2198, 0, 226.0821034, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-10 14:16:35", 28.6327, 77.2198, 0, 226.0821034, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-10 20:51:31", 28.6327, 77.2198, 0, 226.0821034, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 13:00:10", 30.218916, 78.77548, 231.9636831, 458.0457866, 5.778266244, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 15:48:11", 30.223396, 78.81965, 4.273092236, 462.3188788, 1.525952986, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 16:44:36", 30.2233958, 78.8196628, 0.001230027, 462.3201088, 0.001308152, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 16:44:37", 30.2233958, 78.8196628, 0, 462.3201088, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 16:44:37", 30.223396, 78.819664, 0.000117421, 462.3202262, np.inf, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 17:14:56", 30.22339, 78.81966, 0.000769946, 462.3209962, 0.001523808, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 17:50:57", 30.22339, 78.81966, 0, 462.3209962, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 17:50:57", 30.2233958, 78.8196628, 0.000698791, 462.321695, np.inf, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 17:50:57", 30.223396, 78.81965, 0.001230027, 462.322925, np.inf, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 17:50:57", 30.223396, 78.819664, 0.001345122, 462.3242701, np.inf, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-12 20:33:31", 30.223396, 78.819664, 0, 462.3242701, 0, True, "", 0, 0, 0, 0, 0, None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-13 01:36:42", 28.6327, 77.2198, 235.1374861, 697.4617562, 46.53372272, True, "SAFDARJUNG", "(np.float64(28.585), np.float64(77.206))", 5.47240238, 0, 0, "Record of Type 2 but not captured in filtering criteria", None],
    ["0002c72d-ced6-4acf-a11e-95dc3aa6e6d4", "2022-09-13 02:11:05", 30.2233958, 78.8196628, 235.1373929, 932.5991492, 410.3221592, True, "DEHRADUN", "(np.float64(30.19), np.float64(78.18))", 61.58129648, 0, "Y", 0, "Remained same"]
]

# Create the DataFrame
df_sample = pd.DataFrame(data, columns=columns)
df_sample = df_sample[[
    'maid', 'datetime', 'latitude', 'longitude', 'displacement',
    'distance', 'Velocity', 'keep', 
    'Distance to airport (km)', 'Action chosen'
]]
df_sample

maid             datetime   latitude  \
0   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-10 08:10:37  28.632700   
1   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-10 12:04:59  28.632700   
2   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-10 14:10:52  28.632700   
3   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-10 14:16:35  28.632700   
4   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-10 20:51:31  28.632700   
5   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 13:00:10  30.218916   
6   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 15:48:11  30.223396   
7   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 16:44:36  30.223396   
8   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 16:44:37  30.223396   
9   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 16:44:37  30.223396   
10  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 17:14:56  30.223390   
11  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 17:50:57  30.223390   
12  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 17:50:57  30.223396   
13  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 17:50:57  30.223396   
14  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 17:50:57  30.223396   
15  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-12 20:33:31  30.223396   
16  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-13 01:36:42  28.632700   
17  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4  2022-09-13 02:11:05  30.223396   

    longitude  displacement    distance    Velocity  keep  \
0   77.219800      0.000000  226.082103    0.000000  True   
1   77.219800      0.000000  226.082103    0.000000  True   
2   77.219800      0.000000  226.082103    0.000000  True   
3   77.219800      0.000000  226.082103    0.000000  True   
4   77.219800      0.000000  226.082103    0.000000  True   
5   78.775480    231.963683  458.045787    5.778266  True   
6   78.819650      4.273092  462.318879    1.525953  True   
7   78.819663      0.001230  462.320109    0.001308  True   
8   78.819663      0.000000  462.320109    0.000000  True   
9   78.819664      0.000117  462.320226         inf  True   
10  78.819660      0.000770  462.320996    0.001524  True   
11  78.819660      0.000000  462.320996    0.000000  True   
12  78.819663      0.000699  462.321695         inf  True   
13  78.819650      0.001230  462.322925         inf  True   
14  78.819664      0.001345  462.324270         inf  True   
15  78.819664      0.000000  462.324270    0.000000  True   
16  77.219800    235.137486  697.461756   46.533723  True   
17  78.819663    235.137393  932.599149  410.322159  True   

    Distance to airport (km)  Action chosen  
0                   0.000000           None  
1                   0.000000           None  
2                   0.000000           None  
3                   0.000000           None  
4                   0.000000           None  
5                   0.000000           None  
6                   0.000000           None  
7                   0.000000           None  
8                   0.000000           None  
9                   0.000000           None  
10                  0.000000           None  
11                  0.000000           None  
12                  0.000000           None  
13                  0.000000           None  
14                  0.000000           None  
15                  0.000000           None  
16                  5.472402           None  
17                 61.581296  Remained same

In [4]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
from tqdm import tqdm
import numpy as np

def process_dataframe_for_fixing_velocity_recursive(df, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90):
    """
    Process the DataFrame to find and flag records based on given conditions, and adjust datetimes if needed.

    Args:
    - df (DataFrame): The input DataFrame with data.
    - velocity_threshold (float): The threshold for velocity to consider a record.
    - displacement_threshold (float): The threshold for displacement to consider a record.
    - min_velocity (float): The minimum velocity for the ratio of distance to time difference.
    - max_velocity (float): The maximum velocity for the ratio of distance to time difference.

    Returns:
    - DataFrame: The processed DataFrame with records flagged for keeping or removal.
    """
    # Ensure the datetime column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Add new columns for actions chosen and keep flag
    df['Action chosen'] = 'None'
    df['keep'] = True

    # Print initial DataFrame
    print("Initial DataFrame:")
    print(df.head())

    # Calculate geodesic distance
    def calculate_geodesic_distance(lat1, lon1, lat2, lon2, index1, index2):
        distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
        print(f"Index {index1} to Index {index2}: Geodesic distance between ({lat1}, {lon1}) and ({lat2}, {lon2}): {distance} km")
        return distance

    # Calculate time difference in hours
    def calculate_time_diff_hours(time1, time2):
        time_diff = abs((time1 - time2).total_seconds()) / 3600
        print(f"Time difference between {time1} and {time2}: {time_diff} hours")
        return time_diff

    # Recursive function to adjust datetime
    def fix_datetime_recursively(group, index, current_min_velocity, current_max_velocity, max_distance_transition_while_tracing):
        if index == 0:
            return
        valid_record_found = False
        for l in range(index - 1, -1, -1):
            print(f"Processing index {l} in recursive function for index {index}")
            distance_change = calculate_geodesic_distance(
                group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                group.loc[l + 1, 'latitude'], group.loc[l + 1, 'longitude'],
                l, l + 1
            )
            
            if distance_change < max_distance_transition_while_tracing:
                distance = calculate_geodesic_distance(
                    group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                    group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                    index, l
                )
                time_diff_hours = calculate_time_diff_hours(
                    group.loc[index, 'datetime'], group.loc[l, 'datetime']
                )

                velocity_ratio = distance / time_diff_hours if time_diff_hours > 0 else np.inf
                print(f"Velocity ratio for index {index}: {velocity_ratio}")

                if current_min_velocity <= velocity_ratio <= current_max_velocity:
                    print(f"Selecting record at index {l}")
                    group.at[l, 'Action chosen'] = 'selected'
                    group.loc[l + 1:index-1, 'keep'] = False
                    valid_record_found = True
                    break

                elif velocity_ratio < current_min_velocity:
                    distance_l_l1 = calculate_geodesic_distance(
                        group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                        group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                        l + 1, l
                    )
                    time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)

                    distance_l_plus_1_index = calculate_geodesic_distance(
                        group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                        group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                        l + 1, index
                    )
                    new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)

                    if new_time >= time_x:
                        print(f"Modifying datetime at index {l+1} to {new_time}")
                        group.at[l+1, 'datetime'] = new_time
                        group.at[l+1, 'Action chosen'] = 'modified'
                        
                        group.loc[l + 2:index-1, 'keep'] = False
                        valid_record_found = True
                        break
                    else:
                        print(f"Recursively fixing datetime for index {l+1}")
                        fix_datetime_recursively(group, l+1, current_min_velocity, current_max_velocity, max_distance_transition_while_tracing)
            elif distance_change >= max_distance_transition_while_tracing:
                distance_l_l1 = calculate_geodesic_distance(
                    group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                    group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                    l + 1, l
                )
                time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)

                distance_l_plus_1_index = calculate_geodesic_distance(
                    group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                    group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                    l + 1, index
                )
                new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)

                if new_time >= time_x:
                    print(f"Modifying datetime at index {l+1} to {new_time}")
                    group.at[l+1, 'datetime'] = new_time
                    group.at[l+1, 'Action chosen'] = 'modified'
                    
                    group.loc[l + 2:index-1, 'keep'] = False
                    valid_record_found = True
                    break
                else:
                    print(f"Record at index {index} remained same")
                    group.at[index, 'Action chosen'] = 'Remained same'
                    break

        if not valid_record_found:
            print(f"Record at index {index} remained same")
            group.at[index, 'Action chosen'] = 'Remained same'

    # Function to process each group
    def process_group(group):
        max_distance_transition_while_tracing = 20
        group = group.reset_index(drop=True)
        
        relevant_indices = group[(group['Velocity'] > velocity_threshold) & (group['displacement'] > displacement_threshold)].index
        count = len(relevant_indices)

        print(f"Processing group with {len(group)} records, {count} relevant indices found")
        for i in range(count):
            index = relevant_indices[i]
            record = group.iloc[index]
            
            current_min_velocity = min_velocity
            current_max_velocity = max_velocity
            
            if index > 0:
                distance_from_airport_i = group.loc[index, 'Distance to airport (km)']
                distance_from_airport_prev = group.loc[index-1, 'Distance to airport (km)']
                
                if distance_from_airport_i < 2 and distance_from_airport_prev < 2:
                    print(f"Adjusting velocity range due to proximity to airport at index {index}")
                    current_min_velocity = 500
                    current_max_velocity = 600
                    max_distance_transition_while_tracing = 2
                
                fix_datetime_recursively(group, index, current_min_velocity, current_max_velocity, max_distance_transition_while_tracing)
        
        print("Finished processing group")
        return group

    grouped = df.groupby('maid')
    
    processed_groups = []
    for _, group in tqdm(grouped, total=len(grouped), desc="Processing Groups"):
        processed_groups.append(process_group(group))

    df = pd.concat(processed_groups)
    
    # Print final DataFrame
    print("Processed DataFrame:")
    print(df.head())

    return df


In [5]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
from tqdm import tqdm
import numpy as np

def process_dataframe_for_fixing_velocity_recursive(df, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90):
    """
    Process the DataFrame to find and flag records based on given conditions, and adjust datetimes if needed.

    Args:
    - df (DataFrame): The input DataFrame with data.
    - velocity_threshold (float): The threshold for velocity to consider a record.
    - displacement_threshold (float): The threshold for displacement to consider a record.
    - min_velocity (float): The minimum velocity for the ratio of distance to time difference.
    - max_velocity (float): The maximum velocity for the ratio of distance to time difference.

    Returns:
    - DataFrame: The processed DataFrame with records flagged for keeping or removal.
    """
    # Ensure the datetime column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Add new columns for actions chosen and keep flag
    df['Action chosen'] = 'None'
    df['keep'] = True

    # Print initial DataFrame
    # print("Initial DataFrame:")
    # print(df.head())

    # Calculate geodesic distance
    def calculate_geodesic_distance(lat1, lon1, lat2, lon2, index1, index2):
        distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
        print(f"Index {index1} to Index {index2}: Geodesic distance between ({lat1}, {lon1}) and ({lat2}, {lon2}): {distance} km")
        return distance

    # Calculate time difference in hours
    def calculate_time_diff_hours(time1, time2):
        time_diff = abs((time1 - time2).total_seconds()) / 3600
        print(f"Time difference between {time1} and {time2}: {time_diff} hours")
        return time_diff

    # Recursive function to adjust datetime
    def fix_datetime_recursively(group, index, current_min_velocity, current_max_velocity, max_distance_transition_while_tracing):
        if index == 0:
            print(f"Reached the beginning of the group at index {index}, stopping recursion.")
            return
        
        valid_record_found = False
        
        for l in range(index - 2, -1, -1):
            print(f"---\nChecking records between index {l} and {index}")

            distance_change = calculate_geodesic_distance(
                group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                group.loc[l + 1, 'latitude'], group.loc[l + 1, 'longitude'],
                l, l + 1
            )

            print(f"Distance change between index {l} and {l + 1}: {distance_change} km")

            if distance_change < max_distance_transition_while_tracing:
                print(f"Distance change is less than {max_distance_transition_while_tracing} km, checking for velocity adjustment.")

                distance = calculate_geodesic_distance(
                    group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                    group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                    index, l
                )

                time_diff_hours = calculate_time_diff_hours(
                    group.loc[index, 'datetime'], group.loc[l, 'datetime']
                )

                velocity_ratio = distance / time_diff_hours if time_diff_hours > 0 else np.inf
                print(f"Velocity for index {l} compared to {index}: {velocity_ratio} km/h")

                if current_min_velocity <= velocity_ratio <= current_max_velocity:
                    print(f"Velocity ratio is within the range [{current_min_velocity}, {current_max_velocity}].")
                    print(f"Selecting record at index {l}. Marking records {l + 1} to {index - 1} for removal.")
                    
                    group.at[l, 'Action chosen'] = 'selected'
                    group.loc[l + 1:index-1, 'keep'] = False
                    valid_record_found = True
                    break

                elif velocity_ratio < current_min_velocity:
                    print(f"Velocity is below the minimum velocity {current_min_velocity}.")

                    distance_l_l1 = calculate_geodesic_distance(
                        group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                        group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                        l + 1, l
                    )
                    current_avg_velocity = (current_max_velocity + current_min_velocity)/2
                    time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / current_avg_velocity)
                    print(f"Calculated feasible time {time_x} at index {l} for transition from index {l+1}.")

                    distance_l_plus_1_index = calculate_geodesic_distance(
                        group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                        group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                        l + 1, index
                    )

                    new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / current_avg_velocity)
                    print(f"New calculated time for index {index}: {new_time}")

                    if new_time >= time_x:
                        print(f"New time {new_time} is greater than or equal to feasible time {time_x}. Updating datetime at index {l+1}.")

                        group.at[l+1, 'datetime'] = new_time
                        group.at[l+1, 'Action chosen'] = 'modified'
                        
                        group.loc[l + 2:index-1, 'keep'] = False
                        valid_record_found = True
                        break
                    else:
                        print(f"New time {new_time} is less than feasible time {time_x}. Recursively fixing datetime for index {l+1}.")
                        fix_datetime_recursively(group, l+1, current_min_velocity, current_max_velocity, max_distance_transition_while_tracing)
            
            elif distance_change >= max_distance_transition_while_tracing:
                print(f"Distance change is greater than or equal to {max_distance_transition_while_tracing} km, adjusting with max transition.")

                distance_l_l1 = calculate_geodesic_distance(
                    group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                    group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                    l , l+1
                )
                
                current_avg_velocity = (current_max_velocity + current_min_velocity)/2  
                time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / min_velocity)
                print(f"Calculated feasible time {time_x} at index {l+1} for transition from index {l}.")

                distance_l_plus_1_index = calculate_geodesic_distance(
                    group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                    group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                    l + 1, index
                )

                new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / min_velocity)
                print(f"New calculated time for index {l+1}: {new_time}")

                if new_time >= time_x:
                    print(f"New time {new_time} is greater than or equal to feasible time {time_x}. Updating datetime at index {l+1}.")

                    group.at[l+1, 'datetime'] = new_time
                    group.at[l+1, 'Action chosen'] = 'modified'
                    
                    group.loc[l + 2:index-1, 'keep'] = False
                    valid_record_found = True
                    break
                else:
                    print(f"Record at index {index} remained the same due to unresolvable time adjustment.")
                    group.at[index, 'Action chosen'] = 'Remained same'
                    break

        if not valid_record_found:
            print(f"Record at index {index} remained the same after evaluating all possible previous records.")
            group.at[index, 'Action chosen'] = 'Remained same'

    # Function to process each group
    def process_group(group):
        max_distance_transition_while_tracing = 20
        group = group.reset_index(drop=True)
        
        relevant_indices = group[(group['Velocity'] > velocity_threshold) & (group['displacement'] > displacement_threshold)].index
        count = len(relevant_indices)

        print(f"\nProcessing group with {len(group)} records, {count} relevant indices found")
        for i in range(count):
            index = relevant_indices[i]
            record = group.iloc[index]
            
            current_min_velocity = min_velocity
            current_max_velocity = max_velocity
            
            if index > 0:
                distance_from_airport_i = group.loc[index, 'Distance to airport (km)']
                distance_from_airport_prev = group.loc[index-1, 'Distance to airport (km)']
                
                if distance_from_airport_i < 2 and distance_from_airport_prev < 2:
                    print(f"Adjusting velocity range due to proximity to airport at index {index}")
                    current_min_velocity = 500
                    current_max_velocity = 600
                    max_distance_transition_while_tracing = 2
                
                fix_datetime_recursively(group, index, current_min_velocity, current_max_velocity, max_distance_transition_while_tracing)
        
        print("Finished processing group")
        return group

    grouped = df.groupby('maid')
    
    processed_groups = []
    for _, group in tqdm(grouped, total=len(grouped), desc="Processing Groups"):
        processed_groups.append(process_group(group))

    result = pd.concat(processed_groups, ignore_index=True)

    print("Final DataFrame:")
    # print(result.head())

    return result



In [68]:
df_sample

maid            datetime   latitude  \
0   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 08:10:37  28.632700   
1   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 12:04:59  28.632700   
2   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 14:10:52  28.632700   
3   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 14:16:35  28.632700   
4   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 20:51:31  28.632700   
5   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 13:00:10  30.218916   
6   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 15:48:11  30.223396   
7   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:36  30.223396   
8   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:37  30.223396   
9   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:37  30.223396   
10  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:14:56  30.223390   
11  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223390   
12  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
13  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
14  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
15  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 20:33:31  30.223396   
16  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-13 01:36:42  28.632700   
17  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-13 02:11:05  30.223396   

    longitude  displacement    distance    Velocity  keep  \
0   77.219800      0.000000  226.082103    0.000000  True   
1   77.219800      0.000000  226.082103    0.000000  True   
2   77.219800      0.000000  226.082103    0.000000  True   
3   77.219800      0.000000  226.082103    0.000000  True   
4   77.219800      0.000000  226.082103    0.000000  True   
5   78.775480    231.963683  458.045787    5.778266  True   
6   78.819650      4.273092  462.318879    1.525953  True   
7   78.819663      0.001230  462.320109    0.001308  True   
8   78.819663      0.000000  462.320109    0.000000  True   
9   78.819664      0.000117  462.320226         inf  True   
10  78.819660      0.000770  462.320996    0.001524  True   
11  78.819660      0.000000  462.320996    0.000000  True   
12  78.819663      0.000699  462.321695         inf  True   
13  78.819650      0.001230  462.322925         inf  True   
14  78.819664      0.001345  462.324270         inf  True   
15  78.819664      0.000000  462.324270    0.000000  True   
16  77.219800    235.137486  697.461756   46.533723  True   
17  78.819663    235.137393  932.599149  410.322159  True   

    Distance to airport (km) Action chosen  
0                   0.000000          None  
1                   0.000000          None  
2                   0.000000          None  
3                   0.000000          None  
4                   0.000000          None  
5                   0.000000          None  
6                   0.000000          None  
7                   0.000000          None  
8                   0.000000          None  
9                   0.000000          None  
10                  0.000000          None  
11                  0.000000          None  
12                  0.000000          None  
13                  0.000000          None  
14                  0.000000          None  
15                  0.000000          None  
16                  5.472402          None  
17                 61.581296          None

In [70]:
df_sample_result = process_dataframe_for_fixing_velocity_recursive(df_sample, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90)
df_sample_result

Processing Groups: 100%|██████████| 1/1 [00:00<00:00, 500.63it/s]


Processing group with 18 records, 1 relevant indices found
---
Checking records between index 15 and 17
Index 15 to Index 16: Geodesic distance between (30.223396, 78.819664) and (28.6327, 77.2198): 234.9140506200148 km
Distance change between index 15 and 16: 234.9140506200148 km
Distance change is greater than or equal to 20 km, adjusting with max transition.
Index 15 to Index 16: Geodesic distance between (30.223396, 78.819664) and (28.6327, 77.2198): 234.9140506200148 km
Calculated feasible time 2022-09-13 00:28:25.843037 at index 16 for transition from index 15.
Index 16 to Index 17: Geodesic distance between (28.6327, 77.2198) and (30.2233958, 78.8196628): 234.91395714312472 km
New calculated time for index 16: 2022-09-12 22:16:10.162571
Record at index 17 remained the same due to unresolvable time adjustment.
Record at index 17 remained the same after evaluating all possible previous records.
Finished processing group
Final DataFrame:


maid            datetime   latitude  \
0   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 08:10:37  28.632700   
1   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 12:04:59  28.632700   
2   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 14:10:52  28.632700   
3   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 14:16:35  28.632700   
4   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 20:51:31  28.632700   
5   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 13:00:10  30.218916   
6   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 15:48:11  30.223396   
7   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:36  30.223396   
8   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:37  30.223396   
9   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:37  30.223396   
10  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:14:56  30.223390   
11  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223390   
12  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
13  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
14  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
15  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 20:33:31  30.223396   
16  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-13 01:36:42  28.632700   
17  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-13 02:11:05  30.223396   

    longitude  displacement    distance    Velocity  keep  \
0   77.219800      0.000000  226.082103    0.000000  True   
1   77.219800      0.000000  226.082103    0.000000  True   
2   77.219800      0.000000  226.082103    0.000000  True   
3   77.219800      0.000000  226.082103    0.000000  True   
4   77.219800      0.000000  226.082103    0.000000  True   
5   78.775480    231.963683  458.045787    5.778266  True   
6   78.819650      4.273092  462.318879    1.525953  True   
7   78.819663      0.001230  462.320109    0.001308  True   
8   78.819663      0.000000  462.320109    0.000000  True   
9   78.819664      0.000117  462.320226         inf  True   
10  78.819660      0.000770  462.320996    0.001524  True   
11  78.819660      0.000000  462.320996    0.000000  True   
12  78.819663      0.000699  462.321695         inf  True   
13  78.819650      0.001230  462.322925         inf  True   
14  78.819664      0.001345  462.324270         inf  True   
15  78.819664      0.000000  462.324270    0.000000  True   
16  77.219800    235.137486  697.461756   46.533723  True   
17  78.819663    235.137393  932.599149  410.322159  True   

    Distance to airport (km)  Action chosen  
0                   0.000000           None  
1                   0.000000           None  
2                   0.000000           None  
3                   0.000000           None  
4                   0.000000           None  
5                   0.000000           None  
6                   0.000000           None  
7                   0.000000           None  
8                   0.000000           None  
9                   0.000000           None  
10                  0.000000           None  
11                  0.000000           None  
12                  0.000000           None  
13                  0.000000           None  
14                  0.000000           None  
15                  0.000000           None  
16                  5.472402           None  
17                 61.581296  Remained same

In [6]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
from tqdm import tqdm
import numpy as np

def process_dataframe_for_fixing_velocity(df, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90):
    """
    Process the DataFrame to find and flag records based on given conditions, and adjust datetimes if needed.

    Args:
    - df (DataFrame): The input DataFrame with data.
    - velocity_threshold (float): The threshold for velocity to consider a record.
    - displacement_threshold (float): The threshold for displacement to consider a record.
    - min_velocity (float): The minimum velocity for the ratio of distance to time difference.
    - max_velocity (float): The maximum velocity for the ratio of distance to time difference.

    Returns:
    - DataFrame: The processed DataFrame with records flagged for keeping or removal.
    """
    # Ensure the datetime column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Add new columns for actions chosen and keep flag
    df['Action chosen'] = 'None'
    df['keep'] = True

    # Calculate geodesic distance
    def calculate_geodesic_distance(lat1, lon1, lat2, lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).kilometers

    # Calculate time difference in hours
    def calculate_time_diff_hours(time1, time2):
        return abs((time1 - time2).total_seconds()) / 3600

    # Function to process each group
    def process_group(group):
        max_distance_transition_while_tracing = 20
        # Reset index for group processing
        group = group.reset_index(drop=True)
        
        # Filter relevant records based on velocity and displacement thresholds
        relevant_indices = group[(group['Velocity'] > velocity_threshold) & (group['displacement'] > displacement_threshold)].index
        count = len(relevant_indices)
        # print(f'Group size: {len(group)}, Relevant records count: {count}')
        
        # Process each relevant record
        for i in range(count):
            index = relevant_indices[i]
            record = group.iloc[index]
            # print(f'\nProcessing record at index {index}: {record.to_dict()}')
            
            # Set initial velocity range
            current_min_velocity = min_velocity
            current_max_velocity = max_velocity
            current_avg_velocity = (current_min_velocity+current_max_velocity)/2
            
            # Check distance from airport
            if index > 0:
                distance_from_airport_i = group.loc[index, 'Distance to airport (km)']
                distance_from_airport_prev = group.loc[index-1, 'Distance to airport (km)']
                # print(f'Distance from airport (current): {distance_from_airport_i}, (previous): {distance_from_airport_prev}')
                
                if distance_from_airport_i < 2 and distance_from_airport_prev < 2:
                    current_min_velocity = 500
                    current_max_velocity = 600
                    
                    current_avg_velocity = (current_min_velocity+current_max_velocity)/2   
                    max_distance_transition_while_tracing = 2
                    # print(f'Updated velocity range: {current_min_velocity} - {current_max_velocity}')
            
            valid_record_found = False
            # Iterate backwards from index i to find a valid previous record
            for l in range(index - 1, -1, -1):
                if index - l > 1:
                    distance_change = calculate_geodesic_distance(
                        group.loc[l, 'latitude'], group.loc[l, 'longitude'],
                        group.loc[l + 1, 'latitude'], group.loc[l + 1, 'longitude']
                    )
                    # print(f'Distance change between index {l} and {l+1}: {distance_change} km')
                    
                    if distance_change < max_distance_transition_while_tracing:
                        distance = calculate_geodesic_distance(
                            group.loc[index, 'latitude'], group.loc[index, 'longitude'],
                            group.loc[l, 'latitude'], group.loc[l, 'longitude']
                        )
                        time_diff_hours = calculate_time_diff_hours(
                            group.loc[index, 'datetime'], group.loc[l, 'datetime']
                        )
                        # print(f'Distance to previous record: {distance} km, Time difference: {time_diff_hours} hours')
                        
                        if time_diff_hours > 0 and distance >= 0:
                            velocity_ratio = distance / time_diff_hours
                            # print(f'Calculated velocity ratio: {velocity_ratio}')
                        elif time_diff_hours == 0 and distance ==0:
                            velocity_ratio = 0
                            # print(f'Calculated velocity ratio: {velocity_ratio}')                            
                        elif time_diff_hours== 0:
                            velocity_ratio = np.inf
                            # 
                        # print(f'Calculated velocity ratio: {velocity_ratio}')     


                        if current_min_velocity < velocity_ratio < current_max_velocity:
                            group.at[l, 'Action chosen'] = 'selected'
                            group.loc[l + 1:index-1, 'keep'] = False
                            valid_record_found = True
                            # print(f'Record at index {l} selected.')
                            break

                        elif velocity_ratio < current_min_velocity:
                            # Calculate time x for comparison
                            distance_l_l1 = calculate_geodesic_distance(
                                group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                                group.loc[l, 'latitude'], group.loc[l, 'longitude']
                            )
                            time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / current_avg_velocity)
                            # print(f'Time x for record at index {l+1}: {time_x}')

                            # Calculate new time for record l
                            distance_l_plus_1_index = calculate_geodesic_distance(
                                group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                                group.loc[index, 'latitude'], group.loc[index, 'longitude']
                            )
                            new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / current_avg_velocity)
                            # print(f'New time for record at index {l+1}: {new_time}')

                            if new_time >= time_x:
                                group.at[l+1, 'datetime'] = new_time
                                group.at[l+1, 'Action chosen'] = 'modified'
                                
                                # Mark records from l+2 to index-1 for removal
                                group.loc[l + 2:index-1, 'keep'] = False
                                valid_record_found = True
                                # print(f'Record at index {l+1} modified.')
                                break
                    elif distance_change >= max_distance_transition_while_tracing:
                        # print(f'when distance>=20')
                        distance_l_l1 = calculate_geodesic_distance(
                            group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                            group.loc[l, 'latitude'], group.loc[l, 'longitude']
                        )
                        time_x = group.loc[l, 'datetime'] + timedelta(hours=distance_l_l1 / current_avg_velocity)
                        # print(f'Time x for record at index {l+1}: {time_x}')

                        distance_l_plus_1_index = calculate_geodesic_distance(
                            group.loc[l+1, 'latitude'], group.loc[l+1, 'longitude'],
                            group.loc[index, 'latitude'], group.loc[index, 'longitude']
                        )
                        new_time = group.loc[index, 'datetime'] - timedelta(hours=distance_l_plus_1_index / current_avg_velocity)
                        # print(f'New time for record at index {l+1}: {new_time}')
                        if new_time >= time_x:
                            group.at[l+1, 'datetime'] = new_time
                            group.at[l+1, 'Action chosen'] = 'modified'
                            
                            # Mark records from l+2 to index-1 for removal
                            group.loc[l + 2:index-1, 'keep'] = False
                            valid_record_found = True
                            # print(f'Record at index {l+1} modified.')
                            break
                        else:
                            group.at[index, 'Action chosen'] = 'Remained same'
                            break


            if not valid_record_found:
                # If no valid modification was made, mark as 'Remained same'
                group.at[index, 'Action chosen'] = 'Remained same'
                # print(f'Record at index {index} remained the same.')

        return group

    grouped = df.groupby('maid')
    # Process each group with a progress bar
    processed_groups = []
    for _, group in tqdm(grouped, total=len(grouped), desc="Processing Groups"):
        processed_groups.append(process_group(group))

    # Combine the processed groups back into a single DataFrame
    df = pd.concat(processed_groups)

    return df


In [42]:
df_sample_result_1 = process_dataframe_for_fixing_velocity(df_sample, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90)
df_sample_result_1

Processing Groups: 100%|██████████| 1/1 [00:00<00:00, 500.10it/s]

Time x for record at index 16: 2022-09-12 23:41:26.874430
New time for record at index 16: 2022-09-12 23:03:09.130057


maid            datetime   latitude  \
0   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 08:10:37  28.632700   
1   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 12:04:59  28.632700   
2   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 14:10:52  28.632700   
3   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 14:16:35  28.632700   
4   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-10 20:51:31  28.632700   
5   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 13:00:10  30.218916   
6   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 15:48:11  30.223396   
7   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:36  30.223396   
8   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:37  30.223396   
9   0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 16:44:37  30.223396   
10  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:14:56  30.223390   
11  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223390   
12  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
13  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
14  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 17:50:57  30.223396   
15  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-12 20:33:31  30.223396   
16  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-13 01:36:42  28.632700   
17  0002c72d-ced6-4acf-a11e-95dc3aa6e6d4 2022-09-13 02:11:05  30.223396   

    longitude  displacement    distance    Velocity  keep  \
0   77.219800      0.000000  226.082103    0.000000  True   
1   77.219800      0.000000  226.082103    0.000000  True   
2   77.219800      0.000000  226.082103    0.000000  True   
3   77.219800      0.000000  226.082103    0.000000  True   
4   77.219800      0.000000  226.082103    0.000000  True   
5   78.775480    231.963683  458.045787    5.778266  True   
6   78.819650      4.273092  462.318879    1.525953  True   
7   78.819663      0.001230  462.320109    0.001308  True   
8   78.819663      0.000000  462.320109    0.000000  True   
9   78.819664      0.000117  462.320226         inf  True   
10  78.819660      0.000770  462.320996    0.001524  True   
11  78.819660      0.000000  462.320996    0.000000  True   
12  78.819663      0.000699  462.321695         inf  True   
13  78.819650      0.001230  462.322925         inf  True   
14  78.819664      0.001345  462.324270         inf  True   
15  78.819664      0.000000  462.324270    0.000000  True   
16  77.219800    235.137486  697.461756   46.533723  True   
17  78.819663    235.137393  932.599149  410.322159  True   

    Distance to airport (km)  Action chosen  
0                   0.000000           None  
1                   0.000000           None  
2                   0.000000           None  
3                   0.000000           None  
4                   0.000000           None  
5                   0.000000           None  
6                   0.000000           None  
7                   0.000000           None  
8                   0.000000           None  
9                   0.000000           None  
10                  0.000000           None  
11                  0.000000           None  
12                  0.000000           None  
13                  0.000000           None  
14                  0.000000           None  
15                  0.000000           None  
16                  5.472402           None  
17                 61.581296  Remained same

In [51]:
df_all.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'keep', 'Nearest airport', 'Coordinates of airport',
       'Distance to airport (km)', 'Added'],
      dtype='object')

In [16]:
df_all_complete = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\removing_type_3\df_all with airports\df_all_with_airports.csv')

In [7]:
df_all_complete= process_dataframe_for_fixing_velocity(df_all_complete, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90)
df_all_complete.shape

Processing Groups: 100%|██████████| 329076/329076 [10:53<00:00, 503.33it/s]


(39901723, 19)

In [17]:
df_all_complete_with_vel_150_disp_150 = df_all_complete[(df_all_complete['Velocity']>=150) & (df_all_complete['displacement']>=150)]
print(f'df_all_complete_with_vel_150_disp_150 : {df_all_complete_with_vel_150_disp_150.shape}')
count_of_such_ids = df_all_complete_with_vel_150_disp_150['maid'].unique().tolist()
print(f'count_of_such_ids: {len(count_of_such_ids)}')
df_all_complete_with_vel_150_disp_150 = df_all_complete[df_all_complete['maid'].isin(set(count_of_such_ids))]
print(f' TOtal count of records corresponding to these IDs: {len(df_all_complete_with_vel_150_disp_150)}')

df_all_complete_with_vel_150_disp_150 : (589962, 18)
count_of_such_ids: 146079
 TOtal count of records corresponding to these IDs: 30359255


In [10]:
import gc
del df_all_complete_with_vel_150_disp_150
gc.collect()

1883120

In [ ]:
df_all_processed= process_dataframe_for_fixing_velocity(df_all, velocity_threshold=150, displacement_threshold=150, min_velocity=60, max_velocity=90)
df_all_processed.shape

In [14]:
df_all_complete = df_all_complete[df_all_complete['keep']==True]
df_all_complete = optimized_function_maid_vectorized(df_all_complete)

df_all_complete.shape  

(39366340, 19)

In [19]:
df_all_complete.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_without_type_3_without_recursion_meth.csv', index=False)

In [ ]:
df_all_processed = df_all_processed[df_all_processed['keep']==True]
df_all_processed = optimized_function_maid_vectorized(df_all_processed)

df_all_processed.shape  

(3867904, 13)

In [59]:
x= df_all_processed[(df_all_processed['displacement']>=150) & (df_all_processed['Velocity']>=150)]
print(x['maid'].nunique())  # 28375 total records with issue for 21378 IDs

21378
